In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("scrapped_data.csv")

C:\Users\hasin\AppData\Local\Temp\ipykernel_13836\448959118.py:1: DtypeWarning: Columns (185,189,221,253,273) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("scrapped_data.csv")


In [3]:
filtered_df=df[["city","state","zip","baths","beds","fullBaths","lotSize/value","price/value","url","keyFacts/0/description","keyFacts/1/description","keyFacts/2/description","isNewConstruction","isHot","latLong/value/latitude","latLong/value/longitude","listingRemarks"]].copy()

In [4]:
# 1. Rename columns to remove problematic characters
filtered_df.rename(columns={
    'lotSize/value': 'lotSize_value',
    'price/value': 'price_value',
    'keyFacts/0/description': 'keyFacts_0_description',
    'keyFacts/1/description': 'keyFacts_1_description',
    'keyFacts/2/description': 'keyFacts_2_description',
    'latLong/value/latitude': 'latitude',
    'latLong/value/longitude': 'longitude'
}, inplace=True)

In [5]:
# 2. Drop rows where critical small fields are missing ('city', 'price_value')
filtered_df = filtered_df.dropna(subset=['city', 'price_value'])

In [6]:
# 3. Impute numerical fields using median
num_cols = ['baths', 'beds', 'fullBaths', 'lotSize_value']
for col in num_cols:
    median_value = filtered_df[col].median()
    filtered_df[col].fillna(median_value, inplace=True)

C:\Users\hasin\AppData\Local\Temp\ipykernel_13836\9034711.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df[col].fillna(median_value, inplace=True)
C:\Users\hasin\AppData\Local\Temp\ipykernel_13836\9034711.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [7]:
# 5. Fill missing text fields with a placeholder
text_cols = ['keyFacts_0_description', 'keyFacts_1_description', 'keyFacts_2_description', 'listingRemarks']
for col in text_cols:
    filtered_df[col].fillna('Not Provided', inplace=True)

C:\Users\hasin\AppData\Local\Temp\ipykernel_13836\3210211116.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df[col].fillna('Not Provided', inplace=True)


In [8]:
# 6. Verify missing values are handled
df_missing_summary = filtered_df.isnull().sum()

In [9]:
# 1. Combine keyFacts description columns into one
filtered_df['keyFacts_combined'] = (
    filtered_df['keyFacts_0_description'].fillna('') + '; ' +
    filtered_df['keyFacts_1_description'].fillna('') + '; ' +
    filtered_df['keyFacts_2_description'].fillna('')
).str.strip('; ')


In [10]:
# Optionally drop the originals
filtered_df.drop(['keyFacts_0_description', 'keyFacts_1_description', 'keyFacts_2_description'], axis=1, inplace=True)

In [11]:
filtered_df['listingRemarks'] = filtered_df['listingRemarks'].str.strip()

In [12]:
# 7. Save the cleaned dataset
filtered_df.to_csv('filtered_df_cleaned.csv', index=False)

print("Data cleaning complete. Cleaned file saved as 'filtered_df_cleaned.csv'.")
print("Remaining missing values per column:")
print(df_missing_summary)

Data cleaning complete. Cleaned file saved as 'filtered_df_cleaned.csv'.
Remaining missing values per column:
city                      0
state                     0
zip                       0
baths                     0
beds                      0
fullBaths                 0
lotSize_value             0
price_value               0
url                       0
keyFacts_0_description    0
keyFacts_1_description    0
keyFacts_2_description    0
isNewConstruction         0
isHot                     0
latitude                  0
longitude                 0
listingRemarks            0
dtype: int64


In [14]:
filtered_df["url"].isna().sum()

0